In [6]:
import pandas as pd
import numpy as np
import os 
import glob
import torch
from featout_exp import BATCHSIZE, IMAGESIZE
import pathlib

In [7]:
CATS = pathlib.Path("PetImages/Cat")
DOGS = pathlib.Path("PetImages/Dog")

In [8]:
cats = sorted([ i for i in os.listdir(CATS) if ".jpg" in i])
dogs = sorted([ i for i in os.listdir(DOGS) if ".jpg" in i])

cats = pd.Series(cats, name= "Path"  ).to_frame()
cats["Label"] = "Cat"
cats = cats.sample(frac= 1, random_state= 0)
cats["Path"] = "PetImages/Cat/" + cats["Path"]

dogs = pd.Series(dogs, name= "Path"  ).to_frame()
dogs["Label"] = "Dog"
dogs = dogs.sample(frac= 1, random_state= 0)
dogs["Path"] = "PetImages/Dog/" + dogs["Path"]


# get firsthalf

index = 12500//2

first_half = pd.concat((cats.iloc[:index], dogs.iloc[:index])).sample(frac = 1, random_state = 0)
second_half = pd.concat((cats.iloc[index:], dogs.iloc[index:])).sample(frac = 1, random_state = 0)

assert first_half.Path.apply(os.path.exists).all()
assert second_half.Path.apply(os.path.exists).all()

first_half.value_counts("Label"),  second_half.value_counts("Label"),

(Label
 Cat    6250
 Dog    6250
 Name: count, dtype: int64,
 Label
 Cat    6250
 Dog    6250
 Name: count, dtype: int64)

In [11]:
from PIL import Image
import tqdm
 
def can_load_image(path):

        try:
            
            Image.open(str(path)).convert('RGB')
            return True
        except:
            
            return False
    
 
mask = [can_load_image(i) for i in tqdm.tqdm(first_half.Path)]
mask = np.array(mask)

print("missing", (mask == 0).sum())

first_half = first_half.loc[mask]
 
 
mask = [can_load_image(i) for i in tqdm.tqdm(second_half.Path)]
mask = np.array(mask)

print("missing", (mask == 0).sum())

second_half = second_half.loc[mask]

 42%|████▏     | 5259/12500 [00:13<00:16, 433.64it/s]/home/felix/miniconda3/envs/torch/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 12500/12500 [00:30<00:00, 411.44it/s]


missing 1


100%|██████████| 12500/12500 [00:34<00:00, 360.38it/s]

missing 1


In [12]:
first_half.to_csv("csvs/first_half.csv")
second_half.to_csv("csvs/second_half.csv")